### Link gitlab
https://romeogit.univ-reims.fr/mnoizet/visualisationscientifique

#### Implémentation détaillée

1. **volume.cu**  
   - Dupliquer la fonction `mip()` pour créer `minip()` (minimum intensity projection) et `meanip()` (mean intensity projection).
   - Adapter la logique pour chaque fonction :
     - `minip` : cherche la plus petite intensité sur le rayon.
     - `meanip` : calcule la moyenne des intensités valides sur le rayon.
   - Dans `__closesthit__volume_radiance`, utiliser un `switch` sur `optixLaunchParams.frame.renderType` pour appeler la bonne fonction.

2. **LaunchParams.h**  
   - Ajouter `MINIP` et `MEANIP` à l'enum `RENDER_TYPE`.

3. **screenDisplay.cpp**  
   - Ajouter des boutons radio (ligne ~194) pour permettre à l'utilisateur de choisir le mode de rendu (MIP, MINIP, MEANIP) via l'interface ImGui.
   - Modifier la valeur de `parameters->frame.renderType` selon le choix utilisateur.

#### Résultat du filtre MINIP

Affichage du rendu obtenu avec le filtre MINIP :

![Filtre MINIP](capture/MINIP.png)

#### Résultat du filtre MEANIP

Affichage du rendu obtenu avec le filtre MEANIP :

![Filtre MEANIP](capture/MEANIP.png)

#### Code modifié / ajouté

##### VisualisationScientifique/shader/volume.cu
```cuda
// filepath: /home/corentin/Documents/Cours/M2/Projets/RenduVisuLaurantLucas/TP2/VisualisationScientifique/shader/volume.cu
// ...existing code...

  __device__ void minip(){
      // ...voir code complet dans le fichier...
  }

  __device__ void meanip(){
      // ...voir code complet dans le fichier...
  }

  extern "C" __global__ void __closesthit__volume_radiance(){
      switch(optixLaunchParams.frame.renderType) {
        case 0: // MIP
          mip();
          break;
        case 1: // MINIP
          minip();
          break;
        case 2: // MEANIP
          meanip();
          break;
        default:
          mip();
          break;
      }
  }
// ...existing code...
```

##### VisualisationScientifique/common/LaunchParams.h
```cpp
// filepath: /home/corentin/Documents/Cours/M2/Projets/RenduVisuLaurantLucas/TP2/VisualisationScientifique/common/LaunchParams.h
// ...existing code...
enum RENDER_TYPE{MIP, MINIP, MEANIP};
// ...existing code...
```

##### VisualisationScientifique/common/screenDisplay.cpp
```cpp
// filepath: /home/corentin/Documents/Cours/M2/Projets/RenduVisuLaurantLucas/TP2/VisualisationScientifique/common/screenDisplay.cpp
// ...existing code...
    // Ajout des boutons radio pour le type de rendu
    static int renderType = parameters->frame.renderType;
    ImGui::Text("Render Type:");
    ImGui::RadioButton("MIP", &renderType, 0); ImGui::SameLine();
    ImGui::RadioButton("MINIP", &renderType, 1); ImGui::SameLine();
    ImGui::RadioButton("MEANIP", &renderType, 2);
    parameters->frame.renderType = renderType;
// ...existing code...
```